## Data Capture Logics and converting to json line

In [40]:
import pandas as pd
from stepfunctions.inputs import ExecutionInput


In [41]:
!pwd

/home/ec2-user/SageMaker/Xgboost & Linear Learner Notebooks


In [42]:
df = pd.read_csv('s3://ds-mlops-s3/data/scoreoutput/lr/2022/02/17/18/batchscoring.csv.out',header=None)

# Logic for multiple files - TBD

In [43]:
# df = df.loc[:,['year','manufacturer','model','condition','cylinders','fuel','odometer','title_status','transmission','drive','type','paint_color']]

In [44]:
df=df[0:]

In [45]:
df.iloc[:,1:]

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2010.0,5,147,2,4,2,458,0,2,3,8,6,8648.414062
1,2014.0,12,442,0,1,2,687,0,0,1,10,6,9917.359375
2,2006.0,2,518,2,3,2,664,0,0,2,0,1,9494.859375
3,1974.0,5,97,2,1,2,323,0,0,3,9,1,11735.351562
4,2005.0,9,237,0,4,0,75,1,0,0,9,1,10555.460938
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,2015.0,31,462,4,5,2,363,0,0,2,6,6,8501.726562
990,1964.0,25,289,1,4,2,24,0,1,3,2,11,11253.367188
991,2015.0,9,215,4,5,4,565,3,0,0,9,10,8297.640625
992,2012.0,5,428,4,5,4,93,3,0,0,9,6,8554.226562


In [46]:
!aws s3 cp file2 s3://poc-vci-sagemaker/data/scoreinput/custmontdata/fil2.jsonl


The user-provided path file2 does not exist.


In [47]:
#%%writefile datacapture.py
import pandas as pd
import json
import glob
#path = r'/opt/ml/processing/input' # Input path
path='s3://ds-mlops-s3/data/scoreinput/'
#all_files = glob.glob(path + "/*.csv")
all_files=['s3://ds-mlops-s3/data/scoreoutput/lr/2022/02/17/18/batchscoring.csv.out']

counter = 0
print(all_files)
for filename in all_files:
    print("hi")
    df = pd.read_csv(filename,header=None)
    df = df.sample(frac=.25)
    df=df.iloc[:,1:]
    # Create a multiline json
    json_list = json.loads(df.to_json(orient = "records"))
    output_path = "/home/ec2-user/SageMaker/VCI/outputfile"
    print(output_path)
    counter = counter + 1
    data = {}
    data["captureData"]={
            "endpointInput": {
                "observedContentType": "text/csv",
                "mode": "INPUT",
                "data": "132,25,113.2,96,269.9,107,229.1,87,7.1,7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1",
                "encoding": "CSV"
            },
            "endpointOutput": {
                "observedContentType": "text/csv; charset=utf-8",
                "mode": "OUTPUT",
                "data": "6295.23876953125",
                "encoding": "CSV"
            }
        }
    data["eventMetadata"] = {
            "eventId": "",
            "inferenceTime": "2"
        }
    data["eventVersion"] = "0"
    with open(output_path, 'w') as f:
        print(output_path)
        for item in json_list:
            item = list(item.values())
            inpitem = ','.join([str(elem) for elem in item[:-1]])
            #outitem = ','.join([str(elem) for elem in item[-1]])
            outitem=str(item[-1])
            data["captureData"]["endpointInput"]["data"] = inpitem
            data["captureData"]["endpointOutput"]["data"] =outitem
            f.write("%s\n" % data)

['s3://ds-mlops-s3/data/scoreoutput/lr/2022/02/17/18/batchscoring.csv.out']
hi
/home/ec2-user/SageMaker/VCI/outputfile
/home/ec2-user/SageMaker/VCI/outputfile


In [48]:
!aws s3 cp /home/ec2-user/SageMaker/VCI/outputfile s3://ds-mlops-s3/monitoring/

upload: ../VCI/outputfile to s3://ds-mlops-s3/monitoring/outputfile


In [49]:
!aws s3 ls s3://ds-mlops-s3/monitoring/

                           PRE reports/
2021-09-02 19:53:29          0 
2022-02-17 22:14:34       2386 constraints.json
2022-03-17 14:22:55      89228 outputfile
2022-03-16 14:30:41      89212 outputfile.jsonl
2022-02-17 22:14:34     462289 statistics.json


In [50]:
print(output_path)

/home/ec2-user/SageMaker/VCI/outputfile


### Parameter definitions

In [36]:
# Dynamic Job Name
import uuid
monitorjobname = "Batch-model-monitor-{}".format(
    uuid.uuid1().hex
)

In [37]:
jsonpath = "s3://ds-mlops-s3/monitoring/"
statspath = "s3://ds-mlops-s3/monitoring/statistics.json"
constraintspath = "s3://ds-mlops-s3/monitoring/constraints.json"
violationpath = "s3://ds-mlops-s3/monitoring/reports/result/"
# jobname = "$.trainingJobName"
#jobname = execution_input["MonitorJobName"]
jobname = monitorjobname


## Preprocessing container for stats calculation

In [38]:
import boto3

client = boto3.client('sagemaker')

In [39]:
response = client.create_processing_job(
    ProcessingInputs=[
    {
        'InputName': 'input_1',
        'AppManaged': False,
        'S3Input': {
            'LocalPath': '/opt/ml/processing/input/endpoint/poc/AllTraffic/2022/03/10/04',
            'S3Uri': jsonpath, # Changed
            'S3DataDistributionType': 'FullyReplicated',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    }, {
        'InputName': 'baseline',
        'AppManaged': False,
        'S3Input': {
            'LocalPath': '/opt/ml/processing/baseline/stats',
            'S3Uri': statspath,
            'S3DataDistributionType': 'FullyReplicated',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    }, {
        'InputName': 'constraints',
        'AppManaged': False,
        'S3Input': {
            'LocalPath': '/opt/ml/processing/baseline/constraints',
            'S3Uri': constraintspath,
            'S3DataDistributionType': 'FullyReplicated',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
#     }, {
#         'InputName': 'post_processor_script',
#         'AppManaged': False,
#         'S3Input': {
#             'LocalPath': '/opt/ml/processing/code/postprocessing',
#             'S3Uri': 's3://sagemaker-us-east-1-525102048888/sagemaker/DEMO-ModelMonitor/code/postprocessor.py',
#             'S3DataDistributionType': 'FullyReplicated',
#             'S3DataType': 'S3Prefix',
#             'S3InputMode': 'File'
#         }
    },
    ],
    ProcessingOutputConfig={
        'Outputs': [
           {
            'OutputName': 'result',
            'AppManaged': False,
            'S3Output': {
                'LocalPath': '/opt/ml/processing/output',
                'S3Uri': violationpath,
                'S3UploadMode': 'Continuous'
            },
        }
        ],
    },
    ProcessingJobName=jobname,
    ProcessingResources={
        'ClusterConfig': {
            'InstanceCount': 1,
            'InstanceType': 'ml.m5.xlarge',
            'VolumeSizeInGB': 100
        }
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 3600
    },
    AppSpecification={
        'ImageUri': '156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer'
    },
    Environment={
        'baseline_constraints': '/opt/ml/processing/baseline/constraints/constraints.json',
        'baseline_statistics': '/opt/ml/processing/baseline/stats/statistics.json',
        'dataset_format': '{"sagemakerCaptureJson":{"captureIndexNames":["endpointInput","endpointOutput"]}}',
        'dataset_source': '/opt/ml/processing/input/endpoint',
        'end_time': '2022-03-10T05:00:00Z',
        'metric_time': '2022-02-14T04:00:00Z',
        'output_path': '/opt/ml/processing/output',
#         'post_analytics_processor_script': '/opt/ml/processing/code/postprocessing/postprocessor.py',
        'publish_cloudwatch_metrics': 'Disabled',
        'sagemaker_endpoint_name': 'poc',
        'sagemaker_monitoring_schedule_name': 'poc',
        'start_time': '2022-03-07T04:00:00Z'
    },
    RoleArn='arn:aws:iam::014257795134:role/service-role/AmazonSageMaker-ExecutionRole-20210202T172781'

)

In [ ]:
# sec_groups = ["sg-01d629a900f9b4d92"]
# subnets = ["subnet-07bd1dfe6aee76227",
#            "subnet-076950ecc89d4340b",
#            "subnet-0c5a462cb45a14bab"] 

# v_preprocessing_iam_role = "arn:aws:iam::014257795134:role/ds-mlops-sagemaker-role" # IAM role for preprocessing container
